In [1]:
!pip freeze | grep scikit-learn

scikit-learn==1.5.0


In [2]:
!pip install pandas

In [3]:
!python -V

Python 3.9.23


In [4]:
import pickle
import pandas as pd

In [5]:
pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [6]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

In [7]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [9]:
pip install pyarrow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 MB 11.2 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [10]:
df = read_data('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-03.parquet')

In [11]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

In [12]:
import numpy as np

In [13]:
print(np.std(y_pred))

6.247488852238703


In [14]:
year = 2023
month = 3

df['ride_id'] = df.index.to_series().apply(lambda idx: f'{year:04d}/{month:02d}_{idx}')

df_result = pd.DataFrame({
    'ride_id': df['ride_id'],
    'predicted_duration': y_pred
})

output_file = f'predictions_{year:04d}_{month:02d}.parquet'
df_result.to_parquet(output_file, engine='pyarrow', compression=None, index=False)

In [15]:
import os

file_size_bytes = os.path.getsize(output_file)
file_size_mb = file_size_bytes / (1024 * 1024)

In [16]:
print(f"Output file size: {file_size_mb:.2f} MB")

Output file size: 65.46 MB
